# Assemble Program

In [1]:
var x86Assembler = require("../dist/x86Assembler/src/assembler")

In [2]:
// Create Assembler object
var assembler = new x86Assembler.Assembler();

In [3]:
// Define code to be compiled
var code = `
    mov $0xF0,%eax
    mov $0x10,%bl
    mov %bl,(%eax)
`

In [4]:
// Assemble this code 
var assembled = assembler.assembleProgram(code);

// Print out the assembled code in tabular manner
console.log(assembled.toString())

Addr			Machine Code			Assembly
--------------------------------------------------------------------------------
0:			b8 f0 00 00 00                mov  $0xf0,%eax
5:			b3 10                         mov  $0x10,%bl
7:			88 18                         mov  %bl,(%eax)



In [5]:
// Get machine code 
var machineCode = assembled.getMachineCode()
console.log(machineCode)

Uint8Array [
  184, 240,   0,  0, 0,
  179,  16, 136, 24
]


In [6]:
// Get metadat to be used for Debugging in PC
var metadata = assembled.genMetadata()
console.log(metadata)

{
  program: '\n    mov $0xF0,%eax\n    mov $0x10,%bl\n    mov %bl,(%eax)\n',
  line_nums: [ 2, 3, 4 ],
  symbol_table: {},
  reverse_symbol_table: {},
  instruction_start_addr: [ 0, 5, 7 ],
  instruction_lengths: [ 5, 2, 2 ],
  addr_instruction_map: { '0': 0, '5': 1, '7': 2 }
}


# Load Code in PC

In [7]:
var PC = require("../dist/x86PC/src/pc")

In [14]:
// Create PC with 256 Bytes of memory
var pc = new PC.PC(256)

// Some utility functions
function showIP(pc) {
    console.log("Instruction Pointer : 0x" +pc.cpu.getInstructionPointer().toString(16));
}
function showRegister(pc, reg) {
    console.log("Register "+ reg + " : " +pc.cpu.registers[reg].getHexadecimalBytes().join(" "));
}

function showMemory(pc, position, size) {
    console.log("Memory: " + pc.memory.getSlice(position, size));
}

In [15]:
// Load the Machine code in PC starting at address 0
pc.loadAssembledProgram(machineCode, 0)
showIP(pc)

Instruction Pointer : 0x0


In [16]:
// Execute the next instruction
pc.executeNextInstruction()
showIP(pc)
showRegister(pc, "eax")

mov $0xf0, %eax
Instruction Pointer : 0x5
Register eax : 0 0 0 f0


In [17]:
// Execute the next instruction
pc.executeNextInstruction()
showIP(pc)
showRegister(pc, "ebx")

mov $0x10, %bl
Instruction Pointer : 0x7
Register ebx : 0 0 0 10


In [18]:
// Execute the next instruction
pc.executeNextInstruction()
showIP(pc)
showRegister(pc, "ebx")
showMemory(pc, 0xF0 - 2, 6)

mov %bl, 0(%eax)
Instruction Pointer : 0x9
Register ebx : 0 0 0 10
Memory: 0,0,16,0,0,0


In [19]:
// No more instructions
pc.executeNextInstruction()

false